In [1]:
!pip install torch torchvision transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import pandas as pd

url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'
df = pd.read_csv(url)
df = df.iloc[:300]

In [3]:
df.shape

(300, 5)

In [4]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [5]:
!pip install sentence_transformers==2.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.2 MB/s eta 0:00:00


In [6]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
answer_llm = df.iloc[0].answer_llm

# Create embeddings for the answer_llm
embedding_vector = embedding_model.encode(answer_llm)

# Get the first value of the resulting vector
first_value = embedding_vector[0]
print(first_value)

-0.42244658


In [9]:
import numpy as np

answers1 = df['answer_llm'].tolist()
answers2 = df['answer_orig'].tolist()

# Create embeddings and compute dot products
evaluations = []
for answer1, answer2 in zip(answers1, answers2):
    embedding1 = embedding_model.encode(answer1)
    embedding2 = embedding_model.encode(answer2)
    dot_product = np.dot(embedding1, embedding2)
    evaluations.append(dot_product)

# Calculate the 75th percentile
percentile_75 = np.percentile(evaluations, 75)

print("75th Percentile of the score:", percentile_75)

75th Percentile of the score: 31.674307823181152


In [8]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [10]:
# from sentence_transformers import SentenceTransformer
# import numpy as np

# Load the embedding model
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

# Function to normalize a vector
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

answers1 = df['answer_llm'].tolist()
answers2 = df['answer_orig'].tolist()

# Create embeddings, normalize them, and compute cosine similarity
cosine_similarities = []
for answer1, answer2 in zip(answers1, answers2):
    embedding1 = embedding_model.encode(answer1)
    embedding2 = embedding_model.encode(answer2)

    # Normalize the embeddings
    embedding1_norm = normalize_vector(embedding1)
    embedding2_norm = normalize_vector(embedding2)

    # Compute cosine similarity
    cosine_similarity = np.dot(embedding1_norm, embedding2_norm)
    cosine_similarities.append(cosine_similarity)

# Calculate the 75th percentile
percentile_75_cosine = np.percentile(cosine_similarities, 75)

print("75th Percentile of the cosine similarity scores:", percentile_75_cosine)


75th Percentile of the cosine similarity scores: 0.8362347930669785


In [11]:
!pip install rouge

In [12]:
from rouge import Rouge

# Initialize the ROUGE scorer
rouge_scorer = Rouge()

# We get the 11th record since index 10 refers to the 11th row
r = df.iloc[10]

# Compute the ROUGE scores
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

# Extract the F1 score for ROUGE-1
f1_score_rouge1 = scores['rouge-1']['f']

print("F1 score for ROUGE-1:", f1_score_rouge1)


F1 score for ROUGE-1: 0.45454544954545456


In [13]:
# Initialize the ROUGE scorer
rouge_scorer = Rouge()

# We get the 11th record since index 10 refers to the 11th row
r = df.iloc[10]

# Compute the ROUGE scores
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

# Extract the F1 scores for ROUGE-1, ROUGE-2, and ROUGE-L
f1_score_rouge1 = scores['rouge-1']['f']
f1_score_rouge2 = scores['rouge-2']['f']
f1_score_rougeL = scores['rouge-l']['f']

# Calculate the average F1 score
average_f1_score = (f1_score_rouge1 + f1_score_rouge2 + f1_score_rougeL) / 3

print("Average F1 score for ROUGE-1, ROUGE-2, and ROUGE-L:", average_f1_score)


Average F1 score for ROUGE-1, ROUGE-2, and ROUGE-L: 0.35490034990035496


In [14]:
# Initialize the ROUGE scorer
rouge_scorer = Rouge()

rouge_2_scores = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Compute the ROUGE scores for each pair of answers
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]

    # Extract the ROUGE-2 F1 score and add it to the list
    rouge_2_scores.append(scores['rouge-2']['f'])

# Calculate the average ROUGE-2 F1 score
average_rouge_2_f1 = sum(rouge_2_scores) / len(rouge_2_scores)

# Create a DataFrame to store the ROUGE-2 scores
rouge_2_df = pd.DataFrame({'ROUGE-2 F1': rouge_2_scores})

print("Average ROUGE-2 F1 score:", average_rouge_2_f1)


Average ROUGE-2 F1 score: 0.20696501983423318
